# Grid-Q project demonstration

Grid-Q is a project which demonstrates the ability of quantum key distribution to send messages between two different nodes. The project is a collaboration between Argonne National Labs and Oakridge National Lab as an attempt to develop a working QKD simulator on a decentralized energy resource system, where information is exchanged between two power grid nodes. The information exchanged in this simulation mimics the information exchanged in power grid nodes. 


First we will start with importing files from the parent package `sequence`, other packages, and other modules in the grid-q project 

In [16]:
## imports from sequence package
from sequence.kernel.timeline import Timeline
from sequence.components.optical_channel import ClassicalChannel
from eavesdropper_implemented.quantum_channel_eve import QuantumChannelEve
from sequence.qkd.BB84 import pair_bb84_protocols
from sequence.message import Message
from enum import Enum, auto
from sequence.message import Message
import sequence.utils.log as log
import logging
from sequence.constants import MILLISECOND


## package imports
import math
import numpy as np
import json
import onetimepad

## local repo imports 
from message_application_components.performance_metrics.message_accuracy import compare_strings_with_color, count_character_differences
from eavesdropper_implemented.node_GridQ import QKDNode_GridQ
from message_application_components.encryption import otp_encrypt, otp_decrypt
from message_application_components.power_grid_json_generator import load_from_json
from message_application_components.power_grid_csv_generator import write_input_powergrid_csv_file, csv_to_string, string_to_csv

The following class is a class to manage all the keys generated using QKD by storing each nodes key in its own key manager 

In [17]:
## Manages QKD keys
class KeyManager:
    def __init__(self, timeline, keysize, num_keys):
        self.timeline = timeline
        self.lower_protocols = []
        self.keysize = keysize
        self.num_keys = num_keys
        self.keys = []
        self.times = []
        
    def send_request(self):
        for p in self.lower_protocols:
            p.push(self.keysize, self.num_keys) # interface for BB84 to generate key
            
    def pop(self, info): # interface for BB84 to return generated keys
        self.keys.append(info)
        self.times.append(self.timeline.now() * 1e-9)

The following classes are used to format the type of messages being sent using the classical channels which are initialized during the key generation phase

In [18]:
## Defines the message type
class MessageType(Enum):
    REGULAR_MESSAGE = auto()

## Defines the cryptic message exchange protocol
class CrypticMessageExchange:
    def __init__(self, own: "QKDNode_GridQ", another: "QKDNode_GridQ"):
        self.own = own
        self.another = another
        self.messages_sent = []
        self.messages_recieved = []
        self.name = own.name

    def received_message(self, src: str, msg: "Message"):
        """Method to receive encrypts messages.

        Will perform different processing actions based on the message received.

        Args:
            src (str): source node sending message.
            msg (Message): message received.
        """
        assert src == self.another.name
        if msg.msg_type is MessageType.REGULAR_MESSAGE:
            self.own.protocol_stack[0].messages_recieved.append(msg.text)
            self.another.protocol_stack[0].messages_sent.append(msg.text)

## Defines the attributes sent with the message ## TODO: rename, encrypt message
class EncryptedMessage(Message):
    """Message"""
    def __init__(self, msg_type: MessageType, receiver: str, **kwargs):
        super().__init__(msg_type, receiver)
        self.protocol_type = CrypticMessageExchange
        if self.msg_type == MessageType.REGULAR_MESSAGE:
            self.text = kwargs["the_message"]
        else:
            raise Exception("Generated invalid message type {}".format(msg_type))

## The Message Manager

The following class manages the messages which are sent between two different nodes. Each node will have a message manager. 

The message manager has 3 main functions:
* Generating keys using QKD appropriate for messages being sent
* Encrypting and decrypting messages
* Sending messages 

In [19]:
## Class containing QKD messaging abilities
class MessageManager:

    '''
    Code for the request application.

    This application has the capabilities to generate keys and send classical
    encrypted messages using the keys generated using QKD. 

    Attributes:
        own (QKDNode_GridQ): the primary node through which messages are sent and recieved
        another (QKDNode_GridQ): the secondary node which massages are recieved and sent
        keys (List[int]): stores the list of keys generated using QKD
        keys (List[int]): stores the list of keys generated using QKD
        another_message_manager (MessageManager): other node's message manager 
        tl (Timeline): timeline object
        cc0 (ClassicalChannel): classical channel end at node 1
        cc1 (ClassicalChannel): classical channel end at node 2
        qc0 (QuantumChannelEve): quantum channel end at node 1
        qc1 (QuantumChannelEve): quantum channel end at node 2
        km1 (KeyManager): node 1's key manager
        km2 (KeyManager): node 2's key manager
    '''

    def __init__(self, own: "QKDNode_GridQ", another: "QKDNode_GridQ", timeline: Timeline):

        # Nodes
        self.own = own
        self.another = another

        # Key Pool
        self.own_keys = np.empty(0)
        self.another_keys = np.empty(0)

        # Other Node's MessageManager
        self.another_message_manager = None

        # Components
        self.tl = timeline
        self.cc0 = None
        self.cc1 = None
        self.qc0 = None
        self.qc1 = None
        self.km1 = None
        self.km2 = None

        # Message Variables
        self.messages_recieved = []
        self.messages_sent = []

        # Metrics
        self.time_to_generate_keys = None
        self.total_sim_time = 0

    
    def pair_message_manager(self, node):
        self.another_message_manager = node

    def generate_keys(self, keysize , num_keys, internode_distance , attenuation, polarization_fidelity, eavesdropper_eff ):
        """
        Method to generate a specific amount of keys based on the size of the messages

        Parameters:
        sim_time: duration of simulation time (ms)
        keysize: size of generated secure key (bits)
        num_keys: number of keys generated (keys)
        internode_distance: distance between two nodes (m)
        attenuation: attenuation (db/km)
        eavesdropper_efficiency = efficacy of eavesdropper
        """
        # begin by defining the simulation timeline with the correct simulation time

        pair_bb84_protocols(self.own.protocol_stack[0], self.another.protocol_stack[0])
        cc0 = ClassicalChannel("cc_n1_n2", self.tl, distance=internode_distance)
        cc1 = ClassicalChannel("cc_n2_n1", self.tl, distance=internode_distance)
        cc0.set_ends(self.own, self.another.name)
        cc1.set_ends(self.another, self.own.name)
        qc0 = QuantumChannelEve("qc_n1_n2", self.tl, attenuation=attenuation, distance=internode_distance,
                            polarization_fidelity=polarization_fidelity, eavesdropper_efficiency = eavesdropper_eff)
        qc1 = QuantumChannelEve("qc_n2_n1", self.tl, attenuation=attenuation, distance=internode_distance,
                            polarization_fidelity=polarization_fidelity, eavesdropper_efficiency = eavesdropper_eff)
        qc0.set_ends(self.own, self.another.name)
        qc1.set_ends(self.another, self.own.name)


        # instantiate our written keysize protocol
        km1 = KeyManager(self.tl, keysize, num_keys)
        km1.lower_protocols.append(self.own.protocol_stack[0])
        self.own.protocol_stack[0].upper_protocols.append(km1)
        km2 = KeyManager(self.tl, keysize, num_keys)
        km2.lower_protocols.append(self.another.protocol_stack[0])
        self.another.protocol_stack[0].upper_protocols.append(km2)
        
        # start simulation and record timing
        self.tl.init()
        km1.send_request()
        self.tl.run()

        ### setting class variabless
        self.time_to_generate_keys = self.tl.now()
        self.own_keys = np.append(self.own_keys,km1.keys)
        self.another_keys = np.append(self.another_keys,km2.keys)
        self.cc0 = cc0
        self.cc1 = cc1
        self.qc0 = qc0
        self.qc1 = qc1
        self.km1 = km1
        self.km2 = km2
    
    ## Method to generate specific number of keys 
    def customize_keys(self, messages, internode_distance, attenuation, polarization_fidelity, eavesdropper_eff):
        max_length = len(max(messages, key=len))
        num_keys = len(messages)
        max_decimal_number = 10**max_length - 1
        bits_needed = math.ceil(math.log2(max_decimal_number + 1))
        self.generate_keys(bits_needed, num_keys, internode_distance, attenuation, polarization_fidelity, eavesdropper_eff)
 
    ## Method to send messages
    def send_message(self, dst: str, messages: list[str], internode_distance, attenuation, polarization_fidelity, eavesdropper_eff):

        ## Generating right appropriate amount of keys
        self.customize_keys(messages, internode_distance, attenuation, polarization_fidelity, eavesdropper_eff)

        encoded_messages = []
        for i in range(len(messages)):
            # encoded_messages.append(otp_encrypt(messages[i], self.own_keys[i])) ## chat-gpt encrypt
            encoded_messages.append(onetimepad.encrypt(messages[i], str(self.own_keys[i]))) ## onetimepad encryption

        self.own.protocol_stack.clear()
        self.own.protocols.clear()
        self.another.protocol_stack.clear()
        self.another.protocols.clear()

        own_protocol = CrypticMessageExchange(self.own, self.another)
        another_protocol = CrypticMessageExchange(self.another, self.own)

        self.own.protocol_stack.append(own_protocol)
        self.own.protocols.append(own_protocol)
        self.another.protocol_stack.append(another_protocol)
        self.another.protocols.append(another_protocol)

        ## send message
        for i in range(len(encoded_messages)):
            msg = EncryptedMessage(MessageType.REGULAR_MESSAGE, self.another.name, the_message = encoded_messages[i])
            if self.another.name == dst:
                self.own.send_message(self.another.name, msg)
            self.tl.run()

        encrypted_messages_recieved = self.another.protocol_stack[0].messages_recieved
        decrypted_messages_recieved = []
        for i in range(len(messages)):
            # decrypted_messages_recieved.append(otp_decrypt(encrypted_messages_recieved[i], self.another_keys[i])) ## chat-gpt encryption
            decrypted_messages_recieved.append(onetimepad.decrypt(encrypted_messages_recieved[i], str(self.another_keys[i]))) ## onetimepad encryption

        ## Update other message manager
        self.messages_sent = messages

        if self.another_message_manager == None:
            print('Add another message manager')
        else:
            self.another_message_manager.messages_recieved = decrypted_messages_recieved

        ## Message Update
        for i in range(len(messages)):
            compare_strings_with_color(messages[i], decrypted_messages_recieved[i])
            character_error = count_character_differences(messages[i], decrypted_messages_recieved[i])
            print(f'Percent of message with error: {character_error}.')
        
        ## Metrics update
        self.total_sim_time = self.tl.now()
        ## delete used keys
        del self.own_keys
        del self.another_keys

        return self.total_sim_time

: 

The following method intializes the parameters for the simulated hardware

In [5]:
## Testing method
def test(sim_time, msg, internode_distance, attenuation, polarization_fidelity, eavesdropper_eff, backup_qc):
    '''
    Test which simulates sending classical messages using QKD, only using the BB84 protocol
    Parameters:
    sim_time: duration of simulation time (ms)
    msg: list of messages needed to be sent
    keysize: size of generated secure key (bits)
    num_keys: number of keys generated (keys)
    internode_distance: distance between two nodes (m)
    attenuation: attenuation (db/km)
    eavesdropper_efficiency = efficacy of eavesdropper
    '''

    ### Initializes Messanger App
    # timeline initialization
    tl = Timeline(sim_time * 1e9)   


    # node 1 and 2 initialization    
    # Stack size = 1 means only BB84 will be implemented        
    node1 = QKDNode_GridQ("n1", tl, stack_size=1)    
    node1.set_seed(0)                           
    node2 = QKDNode_GridQ("n2", tl, stack_size=1)     
    node2.set_seed(1)

    # if back up quantum channel exists set a back up quantum channel
    if backup_qc:
        qc0_backup = QuantumChannelEve("qc_n1_n2_backup", tl, attenuation=attenuation, distance=internode_distance,
                                polarization_fidelity=1, eavesdropper_efficiency = 0.0)
        qc1_backup = QuantumChannelEve("qc_n2_n1_backup", tl, attenuation=attenuation, distance=internode_distance,
                                polarization_fidelity=1, eavesdropper_efficiency = 0.0)
        node1.set_backup_qchannel(qc0_backup)
        node2.set_backup_qchannel(qc1_backup) 

    # message manager 1 and 2 initialization and pairing
    n1 = MessageManager(node1, node2, tl)
    n2 = MessageManager(node2, node1, tl)
    n1.pair_message_manager(n2)

    ### Sends message to other node
    # Effects: 1) generates keys for node1 and node2 
    # 2) Encrypts messages using node1's keys and sends to node2
    # 3) Node2 decrypts messages using its keys 
    results = n1.send_message('n2', msg, internode_distance, attenuation, polarization_fidelity, eavesdropper_eff)
    return results

## Run the testing code here

Set your parameters for the qkd messaging setup in the first box then run the simulation in the second box. 

In [15]:
# Generating and reading the message file 
write_input_powergrid_csv_file()
filename = './power_grid_datafiles/power_grid_input.csv'
msg_string = csv_to_string(filename)


# Setting up parameters:
simulation_time = 1000                 # (ms)
message = msg_string                   # Data from PowerGridData.json file
internode_distance = 1e3               # Distance between nodes (m)
attenuation = 1e-5                     # (db/m)
classical_channel_delay = MILLISECOND  # TODO: add this as a parameter into the simulation
polarization_fidelity = 1.0            # lower fidelity corresponds with more uncontroled noise in the quantum channel 
eavesdropper_eff = 0.0                 # shows how effective the eavesdropper
backup_qc = True                       # Initalize a back up quantum channel?

Data has been written to './power_grid_datafiles/power_grid_input.csv'


In [13]:
# Running the simulation 

time = test(sim_time = 100, msg = msg_string, internode_distance= 1e3, 
            attenuation = 1e-5, polarization_fidelity = 1, eavesdropper_eff = 0.0, backup_qc = True)  # TODO : the input for msg is string list but I only pass in a string so change that 


print(f'End to end time: {time / 1e9} ms')
print('Created `power_grid_output.csv`')

Key error: 0.0
Sender's (Alice) message: [{"P": "64.198", "Q": "97.126", "V": "84.703", "f": "30.163", "angle": "85.707", "status": "41", "mode": "67"}]
Receiver's (Bob) message: [{"P": "64.198", "Q": "97.126", "V": "84.703", "f": "30.163", "angle": "85.707", "status": "41", "mode": "67"}]
Character Differences: 0
Error Percentage: 0.00%
Percent of message with error: 0.
End to end time: 0.199500001 ms
Created `power_grid_output.csv`
